# Basic Recipe: Static Analysis of a Simple Reform

This is the recipe you should follow first.  Mastering this recipe is a prerequisite for all the other recipes in this cookbook.

**Ingredients**

[Policy reform](http://open-source-economics.github.io/Tax-Calculator/reformA.json) in the `ingredients/reformA.json` file.

*When following the recipe as shown below, you will get several instances 
of the same **ignored** error message from deep inside the Pandas
library that is being used by Tax-Calculator.  After conferring with
the Pandas developers, our expectation is these error messages will go
away when we upgrade to Pandas version 0.22.0, which is scheduled to
be released in January 2018, and which fixes a bug in the Pandas
library.  Meanwhile, the error messages are annoying but harmless.*

## Imports

In [1]:
from __future__ import print_function  # Necessary only if using Python 2.7.
import taxcalc as tc
import pandas as pd
import json
from bokeh.io import show, output_notebook

## Setup

Use publicly-available CPS input file.

NOTE: if you have access to the restricted-use IRS-SOI PUF-based input file
and you have that file (named 'puf.csv') located in the directory
where this script is located, then you can substitute the following
statement for the prior statement:

`recs = Records()`

In [2]:
recs = tc.Records.cps_constructor()

Specify `Calculator` object for static analysis of current-law policy.

In [3]:
pol = tc.Policy()
calc1 = tc.Calculator(policy=pol, records=recs)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


NOTE: `calc1` now contains a PRIVATE COPY of `pol` and a PRIVATE COPY of `recs`,
so we can continue to use `pol` and `recs` in this script without any
concern about side effects from `Calculator` method calls on `calc1`.

Calculate aggregate current-law income tax liabilities for 2018.

In [4]:
calc1.advance_to_year(2018)
calc1.calc_all()
itax_rev1 = calc1.weighted_total('iitax')


In [5]:
print('2018_CLP_itax_rev($B)= {:.2f}'.format(itax_rev1 * 1e-9))

2018_CLP_itax_rev($B)= 1220.29


Read JSON reform file and use (the default) static analysis assumptions.

*Requires `reformA.json` to have been downloaded and put in `ingredients` folder.*

In [6]:
reform_filename = './ingredients/reformA.json'
params = tc.Calculator.read_json_param_objects(reform=reform_filename,
                                               assump=None)
print(f"raw param data: {json.dumps(params, indent=4)}")

raw param data: {
    "policy": {
        "2018": {
            "_STD": [
                [
                    12000,
                    24000,
                    12000,
                    18000,
                    24000
                ]
            ],
            "_STD_Dep": [
                0
            ],
            "_STD_Aged": [
                [
                    0,
                    0,
                    0,
                    0,
                    0
                ]
            ],
            "_II_rt5": [
                0.35
            ],
            "_II_rt6": [
                0.37
            ],
            "_II_rt7": [
                0.42
            ],
            "_PT_rt5": [
                0.35
            ],
            "_PT_rt6": [
                0.37
            ],
            "_PT_rt7": [
                0.42
            ]
        }
    },
    "consumption": {},
    "growdiff_baseline": {},
    "growdiff_response": {}
}


Print reform documentation.

In [7]:
print(tc.Calculator.reform_documentation(params))

REFORM DOCUMENTATION
Baseline Growth-Difference Assumption Values by Year:
none: using default baseline growth assumptions
Policy Reform Parameter Values by Year:
2018:
 _II_rt5 : 0.35
  name: Personal income (regular/non-AMT/non-pass-through) tax rate 5
  desc: The third highest tax rate, applied to the portion of taxable income
        below tax bracket 5 and above tax bracket 4.
  baseline_value: 0.32
 _II_rt6 : 0.37
  name: Personal income (regular/non-AMT/non-pass-through) tax rate 6
  desc: The second higher tax rate, applied to the portion of taxable income
        below tax bracket 6 and above tax bracket 5.
  baseline_value: 0.35
 _II_rt7 : 0.42
  name: Personal income (regular/non-AMT/non-pass-through) tax rate 7
  desc: The tax rate applied to the portion of taxable income below tax
        bracket 7 and above tax bracket 6.
  baseline_value: 0.37
 _PT_rt5 : 0.35
  name: Pass-through income tax rate 5
  desc: The third highest tax rate, applied to the portion of income from 

Implement reform and check for reform error messages.

In [8]:
pol.implement_reform(params['policy'])
if pol.parameter_errors:
    print(f"The policy reform generated the following errors: {pol.parameter_errors}")

## Calculate

Specify Calculator object for static analysis of reform policy.

In [9]:
calc2 = tc.Calculator(policy=pol, records=recs)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


Calculate reform income tax liabilities for 2018.

In [10]:
calc2.advance_to_year(2018)
calc2.calc_all()
itax_rev2 = calc2.weighted_total('iitax')
print('2018_REF_itax_rev($B)= {:.2f}'.format(itax_rev2 * 1e-9))

2018_REF_itax_rev($B)= 1249.73


## Results

Print total revenue estimates for 2018.

*Estimates in billons of dollars rounded to nearest hundredth of a billion.*

In [11]:
print('2018_CLP_itax_rev($B)= {:.2f}'.format(itax_rev1 * 1e-9))
print('2018_REF_itax_rev($B)= {:.2f}'.format(itax_rev2 * 1e-9))

2018_CLP_itax_rev($B)= 1220.29
2018_REF_itax_rev($B)= 1249.73


Generate several other standard results tables.

In [12]:
# aggregate diagnostic tables for cyr
clp_diagnostic_table = calc1.diagnostic_table(1)
ref_diagnostic_table = calc2.diagnostic_table(1)

# income-tax distribution for cyr with CLP and REF results side-by-side
dist_table1, dist_table2 = calc1.distribution_tables(calc2, 'weighted_deciles')
assert isinstance(dist_table1, pd.DataFrame)
assert isinstance(dist_table2, pd.DataFrame)
dist_extract = pd.DataFrame()
dist_extract['funits(#m)'] = dist_table1['s006']
dist_extract['itax1($b)'] = dist_table1['iitax']
dist_extract['itax2($b)'] = dist_table2['iitax']
dist_extract['aftertax_inc1($b)'] = dist_table1['aftertax_income']
dist_extract['aftertax_inc2($b)'] = dist_table2['aftertax_income']

# income-tax difference table by expanded-income decile for cyr
diff_table = calc1.difference_table(calc2, 'weighted_deciles', 'iitax')
assert isinstance(diff_table, pd.DataFrame)
diff_extract = pd.DataFrame()
dif_colnames = ['count', 'tot_change', 'mean',
                'pc_aftertaxinc']
ext_colnames = ['funits(#m)', 'agg_diff($b)', 'mean_diff($)',
                'aftertaxinc_diff(%)']
for dname, ename in zip(dif_colnames, ext_colnames):
    diff_extract[ename] = diff_table[dname]

## Plotting

Generate a decile graph and display it using Bokeh.

In [13]:
fig = calc1.decile_graph(calc2)

In [14]:
output_notebook()

Loading BokehJS ...

In [15]:
show(fig)

## Print tables

CLP diagnostic table for 2018.

In [16]:
clp_diagnostic_table

,2018
Returns (#m),162.860
AGI ($b),10772.526
Itemizers (#m),28.270
Itemized Deduction ($b),763.152
Standard Deduction Filers (#m),134.600
Standard Deduction ($b),2314.591
Personal Exemption ($b),0.000
Taxable Income ($b),8153.662
Regular Tax ($b),1379.356
AMT Income ($b),10234.318


REF diagnostic table for 2018.

In [17]:
ref_diagnostic_table

,2018
Returns (#m),162.860
AGI ($b),10772.526
Itemizers (#m),29.250
Itemized Deduction ($b),781.724
Standard Deduction Filers (#m),133.620
Standard Deduction ($b),2230.969
Personal Exemption ($b),0.000
Taxable Income ($b),8184.432
Regular Tax ($b),1408.941
AMT Income ($b),10223.472


Extract of 2018 distribution tables by baseline expanded-income decile.

*Note: deciles are numbered 0-9 with top decile divided into bottom 5%, 
next 4%, and top 1%, in the lines numbered 11-13, respectively.*

In [18]:
dist_extract

,funits(#m),itax1($b),itax2($b),aftertax_inc1($b),aftertax_inc2($b)
0-10n,0.05,0.004,0.004,-11.895,-11.895
0-10z,0.91,0.000,0.000,0.000,0.000
0-10p,15.33,-4.254,-4.242,159.359,159.348
10-20,16.29,-2.556,-2.534,371.961,371.939
20-30,16.29,1.750,1.785,496.507,496.472
30-40,16.29,7.262,7.336,612.681,612.607
40-50,16.29,15.018,15.213,755.652,755.458
50-60,16.29,26.517,26.813,930.189,929.893
60-70,16.29,52.589,53.086,1144.049,1143.551
70-80,16.29,92.978,93.818,1435.491,1434.652


Extract of 2018 income-tax difference table by expanded-income decile.

In [19]:
diff_extract

,funits(#m),agg_diff($b),mean_diff($),aftertaxinc_diff(%)
0-10n,0.05,0.000,0.0,0.0
0-10z,0.91,0.000,0.0,NaN
0-10p,15.33,0.012,0.8,-0.0
10-20,16.29,0.022,1.4,-0.0
20-30,16.29,0.035,2.2,-0.0
30-40,16.29,0.074,4.5,-0.0
40-50,16.29,0.194,11.9,-0.0
50-60,16.29,0.295,18.1,-0.0
60-70,16.29,0.497,30.5,-0.0
70-80,16.29,0.839,51.5,-0.1
